# Encontrando insights de la UEFA Champions League

In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('uefa')
    .getOrCreate()
)

In [0]:
# Cargar los archivos CSV en DataFrames de PySpark
df_2020 = spark.read.csv("dbfs:/FileStore/uefa_2020.csv", header=True, inferSchema=True)
df_2021 = spark.read.csv("dbfs:/FileStore/uefa_2021.csv", header=True, inferSchema=True)
df_2022 = spark.read.csv("dbfs:/FileStore/uefa_2022.csv", header=True, inferSchema=True)

df_2020.printSchema()
df_2021.printSchema()
df_2022.printSchema()

root
 |-- STAGE: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- PENS: boolean (nullable = true)
 |-- PENS_HOME_SCORE: boolean (nullable = true)
 |-- PENS_AWAY_SCORE: boolean (nullable = true)
 |-- TEAM_NAME_HOME: string (nullable = true)
 |-- TEAM_NAME_AWAY: string (nullable = true)
 |-- TEAM_HOME_SCORE: integer (nullable = true)
 |-- TEAM_AWAY_SCORE: integer (nullable = true)
 |-- POSSESSION_HOME: string (nullable = true)
 |-- POSSESSION_AWAY: string (nullable = true)
 |-- TOTAL_SHOTS_HOME: integer (nullable = true)
 |-- TOTAL_SHOTS_AWAY: integer (nullable = true)
 |-- SHOTS_ON_TARGET_HOME: integer (nullable = true)
 |-- SHOTS_ON_TARGET_AWAY: integer (nullable = true)
 |-- DUELS_WON_HOME: string (nullable = true)
 |-- DUELS_WON_AWAY: string (nullable = true)
 |-- PREDICTION_TEAM_HOME_WIN: string (nullable = true)
 |-- PREDICTION_DRAW: string (nullable = true)
 |-- PREDICTION_TEAM_AWAY_WIN: string (nullable = true)
 |-- LOCATION: string (nullable = true)

root
 |-- S

## TEAM_HOME_WITH_MOST_GOALS

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: `TEAM_NAME_HOME` y `TEAM_HOME_SCORE` ordenadas en orden descendente de `TEAM_HOME_SCORE`. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
# Agrupar por equipo local y sumar los goles anotados en casa (TEAM_HOME_SCORE)
df_2020_filtered = df_2020.filter(df_2020["TEAM_HOME_SCORE"].isNotNull())  

# Agrupar por el equipo local y sumar los goles
df_teams_home_goals = df_2020_filtered.groupBy("TEAM_NAME_HOME").agg(
    F.sum("TEAM_HOME_SCORE").alias("TOTAL_GOALS_HOME")
)

# Ordenar 
top_3_teams_home_goals = df_teams_home_goals.orderBy(F.col("TOTAL_GOALS_HOME").desc()).limit(3)
top_3_teams_home_goals.select("TEAM_NAME_HOME", "TOTAL_GOALS_HOME").show()


+-----------------+----------------+
|   TEAM_NAME_HOME|TOTAL_GOALS_HOME|
+-----------------+----------------+
| Manchester City |              15|
|     Real Madrid |              14|
|   Bayern Munich |              13|
+-----------------+----------------+



## TEAM_WITH_MAJORITY_POSSESSION

2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: `TEAM_NAME` y `GAME_COUNT`, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

> El codigo en resumen primero se asegura de no contener nulos en los datos de la posesion, luego crea una columna donde en base a ese dato guarda el nombre del equipo, ya sea en casa o en visitante. Despues tendriamos una columna con los nombres de los equipos que mas posesion tuvieron por partido, pues esta columna se agrupa por nombres y se cuentan para saber cual es el que mas aparece. Luego mostramos



In [0]:
# Filtrar para identificar
df_2021_filtered = df_2021.filter(
    (df_2021["POSSESSION_HOME"].isNotNull()) & (df_2021["POSSESSION_AWAY"].isNotNull())
)

# Crear una columna que identifica qué equipo tiene la posesión
df_2021_filtered = df_2021_filtered.withColumn(
    "TEAM_WITH_MAJORITY_POSSESSION",
    F.when(F.col("POSSESSION_HOME") > F.col("POSSESSION_AWAY"), F.col("TEAM_NAME_HOME"))
    .otherwise(F.col("TEAM_NAME_AWAY"))
)

# Contar la cantidad de veces que cada equipo tuvo posesión
df_teams_majority_possession = df_2021_filtered.groupBy("TEAM_WITH_MAJORITY_POSSESSION").agg(
    F.count("TEAM_WITH_MAJORITY_POSSESSION").alias("GAME_COUNT")
)

# Ordenar los resultados por la cantidad de veces y coger el equipo más frecuente
top_team_majority_possession = df_teams_majority_possession.orderBy(F.col("GAME_COUNT").desc()).limit(1)

top_team_majority_possession.select("TEAM_WITH_MAJORITY_POSSESSION", "GAME_COUNT").show()

+-----------------------------+----------+
|TEAM_WITH_MAJORITY_POSSESSION|GAME_COUNT|
+-----------------------------+----------+
|                    Liverpool|         9|
+-----------------------------+----------+



## TEAM_WON_DUEL_LOST_GAME_STAGE_WISE

3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
# Equipo local ganó más duelos pero perdió el partido
df_2022_filtered_home = df_2022.filter(
    (df_2022["DUELS_WON_HOME"] > df_2022["DUELS_WON_AWAY"]) & (df_2022["TEAM_HOME_SCORE"] < df_2022["TEAM_AWAY_SCORE"])
)

# Equipo visitante ganó más duelos pero perdió el partido
df_2022_filtered_away = df_2022.filter(
    (df_2022["DUELS_WON_AWAY"] > df_2022["DUELS_WON_HOME"]) & (df_2022["TEAM_AWAY_SCORE"] < df_2022["TEAM_HOME_SCORE"])
)

# Unir ambos
df_teams_lost_game_2022 = df_2022_filtered_home.union(df_2022_filtered_away)

# Seleccionar el STAGE y el equipo que perdió
df_teams_lost_game_result_2022 = df_teams_lost_game_2022.select(
    "STAGE", 
    F.when(F.col("TEAM_HOME_SCORE") < F.col("TEAM_AWAY_SCORE"), F.col("TEAM_NAME_HOME"))
    .otherwise(F.col("TEAM_NAME_AWAY")).alias("TEAM_LOST")
)

df_teams_lost_game_result_2022_sorted.show(truncate=False)

+--------------------------+-------------------+
|STAGE                     |TEAM_LOST          |
+--------------------------+-------------------+
|Group stage: Matchday 6   |Celtic             |
|Group stage: Matchday 6   |Sevilla            |
| Semi-finals second leg   |Milan              |
| Semi-finals second leg   |Real Madrid        |
| Round of 16 second leg   |Liverpool          |
| Round of 16 second leg   |Paris Saint-Germain|
| Round of 16 second leg   |Eintracht Frankfurt|
| Round of 16 second leg   |Leipzig            |
| Round of 16 second leg   |Dortmund           |
| Round of 16 first leg    |Tottenham          |
| Round of 16 first leg    |Chelsea            |
| Round of 16 first leg    |FC Porto           |
| Quarter-finals first leg |Chelsea            |
| Quarter-finals first leg |Napoli             |
| Group stage: Matchday 6  |Salzburg           |
| Group stage: Matchday 6  |Internazionale     |
| Group stage: Matchday 6  |Napoli             |
| Group stage: Match

> A ver, asi como esta arriba se ve un poco lioso porque no te enteras muy bien de que ha pasado verdaderamente, entonces con esto podemos ver un mini resumen de cuantos equipos en cada fase hicieron lo de ganar los duelos y luego perder el partido

In [0]:
# Contar la cantidad de equipos por fase
df_teams_lost_game_result_2022_count = df_teams_lost_game_result_2022_distinct.groupBy("STAGE").count()
df_teams_lost_game_result_2022_count.show(truncate=False)


+--------------------------+-----+
|STAGE                     |count|
+--------------------------+-----+
| Group stage: Matchday 6  |5    |
| Group stage: Matchday 2  |5    |
| Group stage: Matchday 5  |7    |
| Semi-finals second leg   |2    |
| Final                    |1    |
| Round of 16 second leg   |5    |
| Round of 16 first leg    |3    |
| Group stage: Matchday 1  |10   |
|Group stage: Matchday 6   |2    |
| Group stage: Matchday 4  |3    |
| Quarter-finals first leg |2    |
| Group stage: Matchday 3  |11   |
+--------------------------+-----+

